# Projeto: Associação Temporal de Paradas e OPs

Este notebook organiza o pipeline para:

1. Padronizar chaves (preencher `CD_PEDIDO` e `CD_ITEM` a partir de `CD_OP`).
2. Ordenar temporalmente os eventos por máquina, com diagnósticos de monotonicidade.
3. Associar paradas entre ajustes consecutivos.
4. Inferir OP faltante via merge temporal (`asof`).
5. Marcar eventos próximos às trocas de turno.

---
## 📌 Citações do Cliente

> *“Exatamente. Não existe uma associação direta entre evento de Parada e evento de Produção. O sistema basicamente conta chapas e registra eventos de parada.”*

> *“Toda vez que um evento é classificado como Ajuste (cód. 1) o operador informa ao sistema qual será a próxima ordem e automaticamente o sistema encerra a ordem anterior; todas as paradas que estiverem dentro do período entre 2 eventos de Ajuste são automaticamente classificadas como pertencentes àquela OP anterior.”*

> *“Acredito que sejam apontamentos indevidos… Verifique se estes eventos não estão com horários relacionados às trocas de turno (~05:00; 13:00; 21:00).”*

---
## ✅ Vamos implementar todo o pipeline abaixo.


In [1]:
import numpy as np
import pandas as pd

MISSING_TOKENS = {"", "-1", "None", "nan"}

def is_missing_str(s: pd.Series) -> pd.Series:
    s = s.astype("string")
    return s.isna() | s.str.strip().isin(MISSING_TOKENS)

def normalize_datetimes(df, cini="DT_INICIO", cfim="DT_FIM"):
    df = df.copy()
    df[cini] = pd.to_datetime(df[cini], errors='coerce')
    df[cfim] = pd.to_datetime(df[cfim], errors='coerce')
    return df

def split_op_into_cols(df, op_col="CD_OP", ped_col="CD_PEDIDO", item_col="CD_ITEM"):
    df = df.copy()
    df[ped_col] = df.get(ped_col, pd.NA)
    df[item_col] = df.get(item_col, pd.NA)

    op_str = df[op_col].astype("string").str.strip()
    mask = op_str.str.contains("/") & ~op_str.isin(MISSING_TOKENS)

    parts = op_str.where(mask).str.split('/', n=1, expand=True)
    df.loc[mask & is_missing_str(df[ped_col]), ped_col] = parts[0]
    df.loc[mask & is_missing_str(df[item_col]), item_col] = parts[1]

    return df

## ✅ Seção 2 — Ordenação Temporal & Diagnósticos
Aqui detectamos problemas como:
- `BAD_ORDER`: quando `DT_FIM < DT_INICIO` na mesma linha.
- `MONO_BREAK`: quebras de monotonicidade dentro da máquina.


In [2]:
def preparar_tempo_monotonico(df, col_maquina="CD_MAQUINA"):
    df = normalize_datetimes(df)
    df[col_maquina] = df[col_maquina].astype("string")

    df["BAD_ORDER"] = df["DT_FIM"].notna() & df["DT_INICIO"].notna() & (df["DT_FIM"] < df["DT_INICIO"])

    df = df.sort_values([col_maquina, "DT_INICIO", "DT_FIM"], kind="mergesort").reset_index(drop=True)

    df["MONO_BREAK"] = (
        df.groupby(col_maquina)["DT_INICIO"]
          .apply(lambda s: s.diff().dt.total_seconds().fillna(0) < 0)
          .reset_index(level=0, drop=True)
          .astype("int8")
    )

    return df

## ✅ Seção 3 — Associar Paradas entre Ajustes (código 1)
Regra do cliente:
> *Paradas entre dois Ajustes pertencem à OP anterior.*


In [3]:
def associar_paradas_entre_ajustes(df,
                                   col_maquina="CD_MAQUINA",
                                   col_evento="CD_PARADAOUCONV",
                                   col_op="CD_OP",
                                   ajuste_code=1):

    df = df.copy()
    df[col_maquina] = df[col_maquina].astype("string")

    op_atual = None
    last_maq = None

    for idx, row in df.iterrows():
        maq = row[col_maquina]
        cod = pd.to_numeric(row[col_evento], errors='coerce')
        row_op = row[col_op]

        if last_maq is None or maq != last_maq:
            op_atual = None
            last_maq = maq

        if cod == ajuste_code and not is_missing_str(pd.Series([row_op])).iloc[0]:
            op_atual = row_op
            continue

        if cod >= 1 and is_missing_str(pd.Series([row_op])).iloc[0]:
            if op_atual is not None:
                df.at[idx, col_op] = op_atual

    return df

## ✅ Seção 4 — Inferência de OP via Merge Temporal (ASOF)
Regra do cliente:
> *Paradas sem OP devem ser associadas à próxima OP (próximo DT_INICIO) dentro de uma tolerância de 30 min.*


Aqui fazemos o merge **por máquina**, evitando erros de ordenação (`left keys must be sorted`).


In [4]:
def inferir_op_asof(df, tolerancia_min=30):
    df = df.copy()
    df = normalize_datetimes(df)
    df["__IDX__"] = np.arange(len(df))

    cod = pd.to_numeric(df["CD_PARADAOUCONV"], errors='coerce').fillna(0)
    mask_parada = cod >= 1
    mask_sem_op = is_missing_str(df["CD_OP"])

    left = df.loc[mask_parada & mask_sem_op & df["DT_FIM"].notna(), ["__IDX__", "CD_MAQUINA", "DT_FIM"]]
    left = left.rename(columns={"DT_FIM": "KEY_TIME"})

    right = df.loc[(~mask_sem_op) & df["DT_INICIO"].notna(), ["CD_MAQUINA", "DT_INICIO", "CD_OP"]]

    updates = []
    tol = pd.Timedelta(minutes=tolerancia_min)

    for maq, l in left.groupby("CD_MAQUINA"):
        r = right[right["CD_MAQUINA"] == maq]
        if l.empty or r.empty:
            continue

        l = l.sort_values("KEY_TIME")
        r = r.sort_values("DT_INICIO")

        m = pd.merge_asof(
            l,
            r,
            left_on="KEY_TIME",
            right_on="DT_INICIO",
            direction="forward",
            tolerance=tol
        )

        ok = m[m["CD_OP"].notna()]
        for _, row in ok.iterrows():
            updates.append((row["__IDX__"], row["CD_OP"]))

    for idx, op in updates:
        df.loc[df["__IDX__"] == idx, "CD_OP"] = op

    return df.drop(columns=["__IDX__"])

## ✅ Pipeline Final
Executa todas as etapas na sequência sugerida pelo cliente.


In [5]:
def pipeline_paradas_ops(df):

    df = df.copy()

    # 1) Ajustar datas
    df = normalize_datetimes(df)

    # 2) Split OP -> Pedido/Item
    df = split_op_into_cols(df)

    # 3) Ordenação e diagnósticos
    df = preparar_tempo_monotonico(df)

    # 4) Paradas entre ajustes
    df = associar_paradas_entre_ajustes(df)

    # 5) ASOF
    df = inferir_op_asof(df)

    return df

## ✅ Como executar o pipeline

```python
df_tratado = pipeline_paradas_ops(df_merge)
```

✅ Seu dataframe final terá OP preenchida, pedido/item recuperados, e diagnósticos temporais.